##RAG APPLICATION USING TYPESENSE



In [1]:
import typesense

In [2]:
client=typesense.Client({
  'nodes': [{
    'host': '20db7ea6rgyh384fp-1.a2.typesense.net',  # For Typesense Cloud use xxx.a1.typesense.net
    'port': '443',       # For Typesense Cloud use 443
    'protocol': 'https'    # For Typesense Cloud use https
  }],
  'api_key':'90XM5gkStVxB946yuflebYgSQ6xsuu88',
  'connection_timeout_seconds': 2
})

books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string'},
    {'name': 'authors', 'type': 'string[]', 'facet': True},
    {'name': 'publication_year', 'type': 'int32', 'facet': True},
    {'name': 'ratings_count', 'type': 'int32'},
    {'name': 'average_rating', 'type': 'float'}
  ],
  'default_sorting_field': 'ratings_count'
}
print(client.collections.create(books_schema))

Deprecation warning: AnalyticsRulesV1 is deprecated on v30+. Use client.analytics instead.


ObjectAlreadyExists: [Errno 409] A collection with name `books` already exists.

In [3]:
client

In [4]:
with open('books.jsonl', 'r', encoding='utf-8') as jsonl_file:
    data = jsonl_file.read()
    client.collections['books'].documents.import_(data)

Deprecation warning: Overrides is deprecated on v30+. Use client.curation_sets instead.
Deprecation warning: The synonyms API (collections/{collection}/synonyms) is deprecated is removed on v30+. Use synonym sets (synonym_sets) instead.


In [5]:
search_parameters={
    'q':"harry potter",
    'query_by':"title,authors",
    'sort_by':"ratings_count:desc"
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 17,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441102969,
   'text_match_info': {'best_field_score': '2211897868289',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441102969',
    'tokens_matched': 2,
    'typo_prefix_score': 0}},
  {'document': {'authors': ['J.K. Rowling', ' Mary GrandPré', ' R

In [6]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'filter_by' : 'publication_year:<1998',
  'sort_by'   : 'publication_year:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'authors': ['J.K. Rowling', ' Mary GrandPré'],
    'average_rating': 4.44,
    'id': '2',
    'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
    'publication_year': 1997,
    'ratings_count': 4602479,
    'title': "Harry Potter and the Philosopher's Stone"},
   'highlight': {'title': {'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Harry', 'Potter'],
     'snippet': "<mark>Harry</mark> <mark>Potter</mark> and the Philosopher's Stone"}],
   'text_match': 1157451471441102969,
   'text_match_info': {'best_field_score': '2211897868289',
    'best_field_weight': 15,
    'fields_matched': 1,
    'num_tokens_dropped': 0,
    'score': '1157451471441102969',
    'tokens_matched': 2,
    'typo_prefix_score': 0}}],
 'out_of': 9979,
 'page': 1,
 'request_params': {'collection_name

In [7]:
search_parameters = {
  'q'         : 'experyment',
  'query_by'  : 'title',
  'facet_by'  : 'authors',
  'sort_by'   : 'average_rating:desc'
}

client.collections['books'].documents.search(search_parameters)

{'facet_counts': [{'counts': [{'count': 1,
     'highlighted': ' Käthe Mazur',
     'value': ' Käthe Mazur'},
    {'count': 1, 'highlighted': 'Mahatma Gandhi', 'value': 'Mahatma Gandhi'},
    {'count': 1, 'highlighted': 'Gretchen Rubin', 'value': 'Gretchen Rubin'},
    {'count': 1,
     'highlighted': 'James Patterson',
     'value': 'James Patterson'}],
   'field_name': 'authors',
   'sampled': False,
   'stats': {'total_values': 4}}],
 'found': 3,
 'hits': [{'document': {'authors': ['James Patterson'],
    'average_rating': 4.08,
    'id': '569',
    'image_url': 'https://images.gr-assets.com/books/1339277875m/13152.jpg',
    'publication_year': 2005,
    'ratings_count': 172302,
    'title': 'The Angel Experiment'},
   'highlight': {'title': {'matched_tokens': ['Experiment'],
     'snippet': 'The Angel <mark>Experiment</mark>'}},
   'highlights': [{'field': 'title',
     'matched_tokens': ['Experiment'],
     'snippet': 'The Angel <mark>Experiment</mark>'}],
   'text_match': 5787300

In [8]:
# Langchain + Typesense + Groq LLM + RAG Application

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

d:\Gen AI Projects\RAG 1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os
os.environ["GROQ_API_KEY"] = ""

In [10]:
loader = TextLoader("test.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

C:\Users\ABHIK GHOSH\AppData\Local\Temp\ipykernel_16952\13807947.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\ABHIK GHOSH\AppData\Local\Temp\ipykernel_16952\13807947.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 941.90it/s, Materializing param=pooler.dense.weight]                        
MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-b

In [12]:
docsearch=Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        "host": "20db7ea6rgyh384fp-1.a2.typesense.net",  # Use xxx.a1.typesense.net for Typesense Cloud
        "port": "443",  # Use 443 for Typesense Cloud
        "protocol": "https",  # Use https for Typesense Cloud
        "typesense_api_key":"90XM5gkStVxB946yuflebYgSQ6xsuu88",
        "typesense_collection_name": "lang-chain"
    },
    
)

In [13]:
query = "What is artificial intelligence"
found_docs = docsearch.similarity_search(query)
print(found_docs[0].page_content)

Artificial Intelligence (AI) refers to the simulation of human intelligence by computer systems, enabling machines to learn, reason, problem-solve, and make decisions. It is a rapidly advancing technology transforming industries through machine learning, natural language processing, and robotics. From healthcare to finance, AI increases efficiency, although it poses ethical challenges regarding data privacy and job displacement. 
Key details about Artificial Intelligence:
Applications: AI drives advancements in healthcare (diagnosing diseases), agriculture (monitoring crops), and finance (detecting fraud).
Functions: Systems are designed for learning from experience, reasoning, and perception.
Technologies: Key components include machine learning (training algorithms on data), neural networks, and natural language processing.
Benefits: It enhances productivity, allows for automation of tasks, and offers solutions to complex, large-scale data problems.
Challenges: Ethical concerns arise

In [14]:
### Retriever
retriever = docsearch.as_retriever()
retriever

VectorStoreRetriever(tags=['Typesense', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.typesense.Typesense object at 0x0000028D88140710>, search_kwargs={})

In [15]:
query = "Artificial intelligence indepth explanation"
retriever.invoke(query)[0]

Document(metadata={'source': 'test.txt'}, page_content='Artificial Intelligence (AI) refers to the simulation of human intelligence by computer systems, enabling machines to learn, reason, problem-solve, and make decisions. It is a rapidly advancing technology transforming industries through machine learning, natural language processing, and robotics. From healthcare to finance, AI increases efficiency, although it poses ethical challenges regarding data privacy and job displacement. \nKey details about Artificial Intelligence:\nApplications: AI drives advancements in healthcare (diagnosing diseases), agriculture (monitoring crops), and finance (detecting fraud).\nFunctions: Systems are designed for learning from experience, reasoning, and perception.\nTechnologies: Key components include machine learning (training algorithms on data), neural networks, and natural language processing.\nBenefits: It enhances productivity, allows for automation of tasks, and offers solutions to complex, 